In [48]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [49]:
file_list = ['arome_data/arome__001__SP1__00H__2024-08-22T06_00_00Z.grib2', 
             'arome_data/arome__001__SP1__01H__2024-08-22T06_00_00Z.grib2', 
             'arome_data/arome__001__SP1__02H__2024-08-22T06_00_00Z.grib2',
             'arome_data/arome__001__SP1__03H__2024-08-22T06_00_00Z.grib2']
backend_kwargs = {
    'filter_by_keys': {
        'typeOfLevel': 'heightAboveGround',
        'level': 2
    }
}
datasets = [xr.open_dataset(f, engine='cfgrib', backend_kwargs=backend_kwargs) for f in file_list]
ds = xr.concat(datasets, dim='step')

In [50]:
dsf = ds.to_dataframe()
dsf = dsf.reset_index()

In [58]:
mean_dsf = dsf.groupby(['latitude', 'longitude']).mean().reset_index()[['latitude', 'longitude', 't2m']]
mean_dsf['latitude'] = round(mean_dsf['latitude'], 2)
mean_dsf['longitude'] = round(mean_dsf['longitude'], 2)

In [62]:
dep_coords = pd.read_csv('coords_dep_001.csv', low_memory=False)

In [63]:
dep_coords_ain = dep_coords[dep_coords['Dep'] == '01']
dsf_ain = pd.merge(mean_dsf, dep_coords_ain, on=['latitude', 'longitude'], how='inner')

In [65]:
mean_ain = dsf_ain['t2m'].mean()

In [ ]:
lst = []
for i in dep_coords['Dep'].unique():
    dep_coords_dep = dep_coords[dep_coords['Dep'] == i]
    dsf_dep = pd.merge(mean_dsf, dep_coords_dep, on=['latitude', 'longitude'], how='inner')
    mean_dep = dsf_dep['t2m'].mean() - 273.15
    data = [i, mean_dep]
    lst.append(data)
df = pd.DataFrame(lst, columns=['Dep', 'TmpMean'])

In [70]:
df

,Dep,TmpMean
0,01,287.683228
1,02,288.664185
2,03,287.485657
3,04,291.350861
4,05,287.770294
...,...,...
91,93,289.594604
92,94,288.432190
93,95,288.887756
94,2A,295.694061
